In [23]:
import os
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [24]:
# Fungsi untuk ekstraksi fitur PSD
def extract_highest_peak_psd(data, fs, lowcut, highcut, order):
    # ... Kode ekstraksi fitur PSD sesuai dengan kode dasar yang diberikan ...
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')

    # Filter bandpass
    filtered_signal = signal.filtfilt(b, a, data)

    # Rectify signal
    rectified_signal = np.abs(filtered_signal)

    # One-sided FFT
    signal_fft = np.fft.fft(filtered_signal)
    signal_freq = np.fft.fftfreq(len(filtered_signal), 1/fs)
    n = len(filtered_signal)
    freq = signal_freq[0:int(n/2)]
    fft_amp = np.abs(signal_fft)[0:int(n/2)] / n * 2

    # One-sided PSD
    psd = (1/(fs*n)) * np.square(np.abs(signal_fft[:n//2]))
    freq_psd = signal_freq[0:int(n/2)]
    psd_one_side = psd[0:int(n/2)]

    # Ekstraksi fitur highest peak dari PSD
    highest_peak_idx = np.argmax(psd_one_side)
    highest_peak_value = psd_one_side[highest_peak_idx]
    return highest_peak_value

In [39]:
# Fungsi untuk membaca dataset dari folder kelas
def read_data_from_folder(folder_path, class_label, fs, lowcut, highcut, order):
    dataframes = []
    for file in os.listdir(folder_path):
        index=[0]
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        start_index = 1
        end_index = 1551
        df = df[start_index:end_index]
        data = df['Y'].values
        highest_peak_psd = extract_highest_peak_psd(data, fs, lowcut, highcut, order)
        df_highest_peak_psd = pd.DataFrame({'Highest Peak PSD': highest_peak_psd, 'Class': class_label})
        dataframes.append(df_highest_peak_psd)
    return pd.concat(dataframes, ignore_index=True)

# Path ke folder yang berisi dataset untuk masing-masing kelas
label1_folder = 'E:/Pengolahan Data CD Bone-USG/csv-400-400/csv 30%/'
label2_folder = 'E:/Pengolahan Data CD Bone-USG/csv-400-400/csv 50%/'
label3_folder = 'E:/Pengolahan Data CD Bone-USG/csv-400-400/csv 70%/'
label4_folder = 'E:/Pengolahan Data CD Bone-USG/csv-400-400/csv 90%/'

In [42]:
# Parameter filter
fs = 100  # Frekuensi sampling
lowcut = 2.0  # Frekuensi cutoff rendah untuk filter bandpass
highcut = 10.0  # Frekuensi cutoff tinggi untuk filter bandpass
order = 5  # Order filter

In [44]:
# Membaca data untuk masing-masing kelas
label1_data = read_data_from_folder(label1_folder, class_label=0, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
label2_data = read_data_from_folder(label2_folder, class_label=1, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
label3_data = read_data_from_folder(label3_folder, class_label=2, fs=fs, lowcut=lowcut, highcut=highcut, order=order)
label4_data = read_data_from_folder(label4_folder, class_label=3, fs=fs, lowcut=lowcut, highcut=highcut, order=order)

# Menggabungkan data dari masing-masing kelas menjadi satu DataFrame utama
combined_data = pd.concat([label1_data, label2_data, label3_data, label4_data], ignore_index=False)

ValueError: If using all scalar values, you must pass an index

In [ ]:
# Memisahkan fitur highest peak PSD dan label kelas
X = np.array(combined_data['Highest Peak PSD'].values.tolist())
y = np.array(combined_data['Class'].values)


# One-hot encode label kelas
y_encoded = to_categorical(y, num_classes=4)

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Mengubah bentuk data untuk LSTM (tambahkan dimensi waktu)
X_train_lstm = np.expand_dims(X_train, axis=1)
X_test_lstm = np.expand_dims(X_test, axis=1)

In [ ]:
# Membuat model LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Menyimpan model terbaik selama pelatihan
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Melatih model
model.fit(X_train_lstm, y_train, validation_data=(X_test_lstm, y_test), epochs=50, batch_size=16, callbacks=[checkpoint])

# Evaluasi performa model
accuracy = model.evaluate(X_test_lstm, y_test)[1]
print("Akurasi pada data uji:", accuracy)

# Simpan model terbaik untuk digunakan pada data baru
best_model = load_model('best_model.h5')